In [207]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

datadir = '../Collected Data/'
test_participant = 20

def read_file(num):
    dataset = pd.read_csv(datadir+'Collected Labeled Data - Phase 01/participant '+ num +'.csv')
    if 'Time' in dataset.columns:
        dataset.drop('Time', axis=1, inplace=True)
    if 'Start' in dataset.columns:
        dataset.drop('Start', axis=1, inplace=True)
    if 'End' in dataset.columns:
        dataset.drop('End', axis=1, inplace=True)
    return dataset

In [198]:
dataset.head(10)

,Timestamp,X,Y,Z,Magnitude,Pressure,Label
0,0.108582,-0.241821,0.097656,0.965576,1.000176,969.916883,Null
1,0.128479,-0.241821,0.097656,0.965576,1.000176,969.916883,Null
2,0.148438,-0.241821,0.097656,0.965576,1.000176,969.916883,Null
3,0.168518,-0.242676,0.092896,0.965210,0.999576,969.913403,Null
4,0.188507,-0.242676,0.092896,0.965210,0.999576,969.913403,Null
5,0.208466,-0.242676,0.092896,0.965210,0.999576,969.913403,Null
6,0.228577,-0.242676,0.092896,0.965210,0.999576,969.913403,Null
7,0.248352,-0.246338,0.092773,0.971313,1.006349,969.913403,Null
8,0.268463,-0.246338,0.092773,0.971313,1.006349,969.913403,Null
9,0.288574,-0.246338,0.092773,0.971313,1.006349,969.913403,Null


In [199]:
if 'Time' in dataset.columns:
    dataset.drop('Time', axis=1, inplace=True)
if 'Start' in dataset.columns:
    dataset.drop('Start', axis=1, inplace=True)
if 'End' in dataset.columns:
    dataset.drop('End', axis=1, inplace=True)

In [208]:
import numpy as np

def calculate_slope(data):
    data = data.replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna()
    x = np.arange(len(data))
    if len(x)>1:
        slope = np.polyfit(x, data, 1)[0]
    else: 
        slope = 0
    return slope

In [209]:
def summarize_interval(group):
    summary = {
        'avg_accX': group['X'].mean(),
        'min_accX': group['X'].min(),
        'max_accX': group['X'].max(),
        'var_accX': group['X'].var(),
        'std_accX': np.std(group['X']),
        'avg_accY': group['Y'].mean(),
        'min_accY': group['Y'].min(),
        'max_accY': group['Y'].max(),
        'var_accY': group['Y'].var(),
        'std_accY': np.std(group['Y']),
        'avg_accZ': group['Z'].mean(),
        'min_accZ': group['Z'].min(),
        'max_accZ': group['Z'].max(),
        'var_accZ': group['Z'].var(),
        'std_accZ': np.std(group['Z']),
        'avg_magnitude': group['Magnitude'].mean(),
        'min_magnitude': group['Magnitude'].min(),
        'max_magnitude': group['Magnitude'].max(),
        'var_magnitude': group['Magnitude'].var(),
        'std_magnitude': np.std(group['Magnitude']),
        'avg_pressure': group['Pressure'].mean(),
        'min_pressure': group['Pressure'].min(),
        'max_pressure': group['Pressure'].max(),
        'var_pressure': group['Pressure'].var(),
        'range_pressure': (group['Pressure'].max() - group['Pressure'].min()),
        'std_pressure': np.std(group['Pressure']),
        'slope_pressure': calculate_slope(group['Pressure']),
        'kurtosis_pressure': kurtosis(group['Pressure']),
        'Interqartile_percentage_Pressure': (np.percentile(group['Pressure'], 75) - np.percentile(group['Pressure'], 25)),
        'skew_pressure': skew(group['Pressure']),
        'Label': group['Label'].mode().iloc[0]  # Most frequent label in the interval
    }
    return pd.Series(summary)

In [215]:
def preprocessing(num, dataset):
    # Applying the summarize_interval function to each 2-second interval
    interval_seconds = 2
    result_df = dataset.groupby(dataset['Timestamp'] // interval_seconds).apply(summarize_interval)
    
    # Reset index and drop the Timestamp column (if needed)
    result_df.reset_index(drop=True, inplace=True)
    
    # Step 5: Save the preprocessed data to a new CSV file
    result_df.to_csv(datadir+'preprocessed/'+str(test_participant)+'/preprocessed_data'+str(num)+'.csv', index=False)
    result_df.head(24)

In [216]:
import glob

for i in range(1, 21):
    if i != test_participant:
        preprocessing(i, read_file(f"0{i}" if i < 9 else str(i)))

dir = (datadir+'preprocessed/'+str(test_participant)+'/')
csv_files = glob.glob(dir+'*.csv')
print(len(csv_files))

C:\Users\NiMou\AppData\Local\Temp\ipykernel_21664\1348150413.py:30: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'kurtosis_pressure': kurtosis(group['Pressure']),
C:\Users\NiMou\AppData\Local\Temp\ipykernel_21664\1348150413.py:32: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'skew_pressure': skew(group['Pressure']),


19


In [212]:
dfs = []

csv_files = csv_files[:19]
# Loop through each CSV file and append its DataFrame to the list
for file in csv_files:
    df = pd.read_csv(file, converters={'Label': lambda x: x.lower() if x not in ['Null', 'null', ' null'] else x})
    dfs.append(df)
    print(file)
# Concatenate all DataFrames in the list along rows (axis=0)
merged_df = pd.concat(dfs, ignore_index=True)


../Collected Data/preprocessed/20\preprocessed_data1.csv
../Collected Data/preprocessed/20\preprocessed_data10.csv
../Collected Data/preprocessed/20\preprocessed_data11.csv
../Collected Data/preprocessed/20\preprocessed_data12.csv
../Collected Data/preprocessed/20\preprocessed_data13.csv
../Collected Data/preprocessed/20\preprocessed_data14.csv
../Collected Data/preprocessed/20\preprocessed_data15.csv
../Collected Data/preprocessed/20\preprocessed_data16.csv
../Collected Data/preprocessed/20\preprocessed_data17.csv
../Collected Data/preprocessed/20\preprocessed_data18.csv
../Collected Data/preprocessed/20\preprocessed_data19.csv
../Collected Data/preprocessed/20\preprocessed_data2.csv
../Collected Data/preprocessed/20\preprocessed_data3.csv
../Collected Data/preprocessed/20\preprocessed_data4.csv
../Collected Data/preprocessed/20\preprocessed_data5.csv
../Collected Data/preprocessed/20\preprocessed_data6.csv
../Collected Data/preprocessed/20\preprocessed_data7.csv
../Collected Data/pre

In [217]:
merged_df.to_csv(datadir+'preprocessed/'+str(test_participant)+'/preprocessed_traindata.csv', index=False)

In [214]:
preprocessing(test_participant, read_file(f"0{test_participant}" if test_participant < 9 else str(test_participant)))
newdf = pd.read_csv(datadir+'preprocessed/'+str(test_participant)+'/preprocessed_data'+str(test_participant)+'.csv', converters={'Label': lambda x: x.lower() if x not in ['Null', 'null', ' null'] else x})
newdf.to_csv(datadir+'preprocessed/'+str(test_participant)+'/preprocessed_testdata.csv', index=False)